In [6]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

def load_and_prepare_image(image_path, target_size):
    """
    Loads and preprocesses an image for the model.

    Args:
        image_path (str): Path to the image file.
        target_size (tuple): Target size to resize the image (width, height).

    Returns:
        np.array: Preprocessed image ready for model prediction.
    """
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Check if the image was successfully loaded
    if image is None:
        raise ValueError(f"Could not load the image at path: {image_path}")

    # Resize the image to the target size
    image_resized = cv2.resize(image, target_size)

    # Normalize pixel values to [0, 1]
    image_normalized = image_resized / 255.0

    # Convert to a NumPy array and add a batch dimension
    image_array = img_to_array(image_normalized)
    image_array = np.expand_dims(image_array, axis=0)  # Shape: (1, height, width, channels)

    return image_array

def predict_image(image_path, model_path, target_size=(128, 128)):
    """
    Loads the model, preprocesses the image, and makes a prediction.

    Args:
        image_path (str): Path to the input image.
        model_path (str): Path to the saved model.
        target_size (tuple): Target size for resizing the image (width, height).

    Returns:
        str: Predicted class label.
        float: Confidence score for the prediction.
    """
    # Load the model
    model = load_model(model_path)
    print("Model loaded successfully.")

    # Load and preprocess the image
    image_array = load_and_prepare_image(image_path, target_size)
    print(f"Image loaded and preprocessed: {image_array.shape}")

    # Make a prediction
    prediction = model.predict(image_array)[0][0]  # Get the first prediction
    print(f"Raw Prediction: {prediction}")

    # Interpret the prediction
    if prediction > 0.5:
        return "Non-Defective", prediction
    else:
        return "Defective", prediction

# Test the function
if __name__ == "__main__":
    # Replace these with your file paths
    image_path = r"D:\Cologne\Sample DataSet 4\train\non-defective\IMG_20241216_142429.jpg"
    model_path = r"my_model_3.0.h5"

    try:
        label, confidence = predict_image(image_path, model_path)
        print(f"Prediction: {label} (Confidence: {confidence:.2f})")
    except Exception as e:
        print(f"Error: {e}")


Model loaded successfully.
Image loaded and preprocessed: (1, 128, 128, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 948ms/step
Raw Prediction: 0.7972398400306702
Prediction: Non-Defective (Confidence: 0.80)


In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

def process_frame(frame, target_size):
    """
    Processes a webcam frame for model prediction.
    
    Args:
        frame (np.array): Captured frame from the webcam.
        target_size (tuple): Target size for resizing the frame (width, height).

    Returns:
        np.array: Preprocessed image ready for model prediction.
    """
    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

def predict_frame(frame, model, target_size):
    """
    Predicts the class of a single frame using the model.

    Args:
        frame (np.array): Captured frame from the webcam.
        model: Loaded Keras model.
        target_size (tuple): Target size for resizing the frame.

    Returns:
        str: Predicted class label.
        float: Confidence score.
    """
    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

    # Make a prediction
    prediction = model.predict(frame_array)[0][0]  # Get the confidence score
    print(f"Raw Prediction: {prediction}")

    # Interpret the prediction
    if prediction > 0.5:
        return "Non-Defective", prediction
    else:
        return "Defective", prediction

def run_webcam_with_model(model_path, target_size=(128, 128)):
    """
    Opens the webcam, captures frames, and processes them using the model.

    Args:
        model_path (str): Path to the saved Keras model.
        target_size (tuple): Target size for resizing frames (width, height).
    """
    # Load the model
    print("Loading model...")
    model = load_model(model_path)
    print("Model loaded successfully.")

    # Open the webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Cannot open the webcam")

    print("Webcam opened. Press 'ESC' to exit.")
    try:
        while True:
            # Capture frame from the webcam
            ret, frame = cap.read()
            if not ret:
                print("Failed to capture frame. Exiting...")
                break

            # Process and predict
            feedback, confidence = predict_frame(frame, model, target_size)
            print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

            # Display feedback on the frame
            cv2.putText(frame, f"Prediction: {feedback}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Confidence: {confidence:.2f}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Show the frame in a window
            cv2.imshow("Webcam Feed", frame)

            # Exit on 'ESC' key
            if cv2.waitKey(1) & 0xFF == 27:  # 27 is the ASCII code for ESC
                print("Exiting...")
                break

    finally:
        # Release resources
        cap.release()
        cv2.destroyAllWindows()

# Run the script
if __name__ == "__main__":
    model_path = "my_model_3.0.h5"  # Path to your saved model
    run_webcam_with_model(model_path, target_size=(128, 128))


Loading model...


Model loaded successfully.
Webcam opened. Press 'ESC' to exit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 825ms/step
Raw Prediction: 0.3893764317035675
Prediction: Defective (Confidence: 0.39)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Raw Prediction: 0.0951802134513855
Prediction: Defective (Confidence: 0.10)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Raw Prediction: 0.07949624210596085
Prediction: Defective (Confidence: 0.08)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Raw Prediction: 0.022040722891688347
Prediction: Defective (Confidence: 0.02)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Raw Prediction: 0.022040722891688347
Prediction: Defective (Confidence: 0.02)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Raw Prediction: 0.17191436886787415
Prediction: Defective (Confidence: 0.17)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Raw Prediction: 0.5402867197990417
Prediction: Non-Defective (Confidence: 0.54)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Raw Prediction: 0.5121685266494751
Prediction: Non-Defective (Confidence: 0.51)
1/1 ━━━━━━

In [1]:
import time

def run_webcam_with_model(model_path, target_size=(128, 128), interval=1):
    """
    Opens the webcam, captures frames every `interval` seconds, and processes them using the model.

    Args:
        model_path (str): Path to the saved Keras model.
        target_size (tuple): Target size for resizing frames (width, height).
        interval (int): Time interval (in seconds) between captures.
    """
    # Load the model
    print("Loading model...")
    model = load_model(model_path)
    print("Model loaded successfully.")

    # Open the webcam
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        raise IOError("Cannot open the webcam")

    print(f"Webcam opened. Capturing every {interval} second(s). Press 'ESC' to exit.")
    try:
        while True:
            # Capture frame from the webcam
            ret, frame = cap.read()
            if not ret:
                print("Failed to capture frame. Exiting...")
                break

            # Process and predict
            feedback, confidence = predict_frame(frame, model, target_size)
            print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

            # Display feedback on the frame
            cv2.putText(frame, f"Prediction: {feedback}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Confidence: {confidence:.2f}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Show the frame in a window
            cv2.imshow("Webcam Feed", frame)

            # Wait for 'interval' seconds
            time.sleep(interval)

            # Exit on 'ESC' key
            if cv2.waitKey(1) & 0xFF == 27:  # 27 is the ASCII code for ESC
                print("Exiting...")
                break

    finally:
        # Release resources
        cap.release()
        cv2.destroyAllWindows()

# Run the script
if __name__ == "__main__":
    model_path = "my_model_4.0.h5"  # Path to your saved model
    run_webcam_with_model(model_path, target_size=(128, 128), interval=1)  # Capture every second


Loading model...


NameError: name 'load_model' is not defined